## 4 - Network Structure
How many neurons are simulated? 

How many degrees of freedom (weights) does the network have?

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from numpy.random import choice, random_integers
from mnist import MNIST
import tensorflow as tf


data = MNIST()
BATCHSIZE = 100
EPOCHS = 10000
TESTSTEPSIZE = 100

## Model

In [ ]:
x = tf.placeholder(tf.float32, (None, 28, 28), name='input')
t = tf.placeholder(tf.int64, (None), name='target')
lr = tf.placeholder(tf.float32, (None), name='learning_rate')

w0 = tf.Variable(tf.truncated_normal((6,6,1,6), stddev=0.1), name='kernel')
b0 = tf.Variable(tf.ones((6,))/10, name='biases')
c0 = tf.nn.conv2d(tf.reshape(x, (-1,28,28,1)), w0, (1,1,1,1), 'SAME', name='convolution')
y0 = tf.nn.relu(c0 + b0, name='activation')

w1 = tf.Variable(tf.truncated_normal((5,5,6,12), stddev=0.1), name='kernel')
b1 = tf.Variable(tf.ones((12,))/10, name='biases')
c1 = tf.nn.conv2d(y0, w1, (1,2,2,1), 'SAME', name='convolution')
y1 = tf.nn.relu(c1 + b1, name='activation')

w2 = tf.Variable(tf.truncated_normal((4,4,12,24), stddev=0.1), name='kernel')
b2 = tf.Variable(tf.ones((24,))/10, name='biases')
c2 = tf.nn.conv2d(y1, w2, (1,2,2,1), 'SAME', name='convolution')
y2 = tf.nn.relu(c2 + b2, name='activation')

w3 = tf.Variable(tf.truncated_normal((7*7*24,200), stddev=0.1), name='readout_hidden_weights')
b3 = tf.Variable(tf.ones((200,))/10, name='readout_hidden_biases')
y3 = tf.nn.relu(tf.matmul(tf.reshape(y2, (-1,7*7*24)), w3) + b3, name='readout_hidden')

wo = tf.Variable(tf.truncated_normal((200,10), stddev=0.1), name='weights')
bo = tf.Variable(tf.zeros((10,)), name='biases')
y = tf.matmul(y3, wo) + bo

entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, t)
entropy = tf.reduce_mean(entropy) * 100
train_step = tf.train.AdamOptimizer(lr).minimize(entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), t)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Training

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

rate_max = 0.003
rate_min = 0.0001
rate_decay = 2000

for i in range(1, EPOCHS + 1):
    batch_x, batch_t = data.getTrainingBatch(10)
    rate = rate_min + (rate_max - rate_min) * np.exp(-i/rate_decay)
    
    sess.run(train_step, {x: batch_x, t: batch_t, lr: rate})
    if i % 100 == 0:
        acc = sess.run(accuracy, {x: data.testData, t: data.testLabels})
        print(i, acc, rate)